In [1]:
from src.OpenAIUtils import get_message
from src.XMLUtils import extract_sections, parse_xml_file, run_grobid
from src.translator import create_llama_cpp_model

In [2]:
# dir_path, _, pdf_name = get_pdf("2103.00020")
dir_path = "/home/paper_translator/data/documents/Learning_Transferable_Visual_Models_From_Natural_Language_Supervision"
pdf_name = (
    "Learning_Transferable_Visual_Models_From_Natural_Language_Supervision"
)

In [3]:
run_grobid(dir_path, pdf_name)
xml_path = dir_path + "/" + pdf_name + ".tei.xml"
root = parse_xml_file(xml_path)
sections = extract_sections(root=root)

9月 25, 2023 8:12:11 午後 org.grobid.core.main.GrobidHomeFinder getGrobidHomePathOrLoadFromClasspath
情報: No Grobid property was provided. Attempting to find Grobid home in the current directory...
9月 25, 2023 8:12:11 午後 org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
情報: ***************************************************************
9月 25, 2023 8:12:11 午後 org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
情報: *** USING GROBID HOME: /usr/lib/grobid-0.7.3/grobid-home
9月 25, 2023 8:12:11 午後 org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
情報: ***************************************************************
9月 25, 2023 8:12:11 午後 org.grobid.core.main.GrobidHomeFinder getGrobidHomePathOrLoadFromClasspath
情報: No Grobid property was provided. Attempting to find Grobid home in the current directory...
9月 25, 2023 8:12:11 午後 org.grobid.core.main.GrobidHomeFinder findGrobidHomeOrFail
情報: ***************************************************************
9月 25, 2023 8:12:11

Processing: /home/paper_translator/data/documents/Learning_Transferable_Visual_Models_From_Natural_Language_Supervision/Learning_Transferable_Visual_Models_From_Natural_Language_Supervision.pdf


9月 25, 2023 8:12:11 午後 org.grobid.core.main.LibraryLoader load
情報: Loading external native sequence labelling library
9月 25, 2023 8:12:11 午後 org.grobid.core.main.LibraryLoader load
情報: Loading Wapiti native library...
9月 25, 2023 8:12:11 午後 org.grobid.core.main.LibraryLoader load
情報: Native library for sequence labelling loaded
9月 25, 2023 8:12:11 午後 org.grobid.core.lexicon.Lexicon initDictionary
情報: Initiating dictionary
9月 25, 2023 8:12:11 午後 org.grobid.core.lexicon.Lexicon initDictionary
情報: End of Initialization of dictionary
9月 25, 2023 8:12:11 午後 org.grobid.core.lexicon.Lexicon initNames
情報: Initiating names
9月 25, 2023 8:12:11 午後 org.grobid.core.lexicon.Lexicon initNames
情報: End of initialization of names
9月 25, 2023 8:12:12 午後 org.grobid.core.lexicon.Lexicon initCountryCodes
情報: Initiating country codes
9月 25, 2023 8:12:12 午後 org.grobid.core.lexicon.Lexicon initCountryCodes
情報: End of initialization of country codes
9月 25, 2023 8:12:12 午後 org.grobid.core.jni.WapitiModel init
情報


************************************************************************************
COUNTER: org.grobid.core.engines.counters.TableRejectionCounters
************************************************************************************
------------------------------------------------------------------------------------
  EMPTY_LABEL_OR_HEADER_OR_CONTENT:   15
  HEADER_NOT_STARTS_WITH_TABLE_WORD:  1
  HEADER_AND_CONTENT_DIFFERENT_PAGES: 1
  HEADER_NOT_CONSECUTIVE:             1
  HEADER_AND_CONTENT_INTERSECT:       2

************************************************************************************
COUNTER: org.grobid.core.engines.counters.ReferenceMarkerMatcherCounters
************************************************************************************
------------------------------------------------------------------------------------
  MATCHED_REF_MARKERS_AFTER_POST_FILTERING: 65
  UNMATCHED_REF_MARKERS:                    2
  STYLE_AUTHORS:                            253
  MANY_C

In [4]:
llm_model = create_llama_cpp_model(
    package_name="langchain",
    model_path="/home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf",
    temperature = 0.1,
)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /home/paper_translator/data/models/ELYZA-japanese-Llama-2-7b-fast-instruct-q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 45043,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.f

In [5]:
section = sections[0]
print(section)

In [6]:
SYSTEM = """
### 指示 ###
文章の内容の中で、重要なポイントを3つ箇条書きしてください。
箇条書きは、文章の内容を簡潔にまとめたものである必要があります。
箇条書きは、以下の制約に従ってください。

### 箇条書きの制約 ###
- 最大5個まで
- 箇条書き1個を50文字以内

### 対象とする論文の内容 ###
{text}

### 出力形式 ###
- 箇条書き1
- 箇条書き2
- 箇条書き3
"""
response = get_message(text=section.body, system=SYSTEM)

In [7]:
print(response)

- Pre-training methods directly from raw text have revolutionized NLP, and there is potential for a similar breakthrough in computer vision.
- Natural language supervision for image representation learning is still rare due to lower performance compared to alternative approaches, but well-targeted uses of weak supervision have improved performance.
- CLIP is an efficient method of learning from natural language supervision and can be competitive with prior task-specific supervised models, with significant policy and ethical implications.


In [8]:
from src.translator import summarize_translator

In [9]:
summarizer = summarize_translator(llm_model=llm_model, max_tokens=60)

In [31]:
temp_file = "/home/paper_translator/data/prompt_temp/prompt_test.txt"
summarizer.create_prompr_from_file(temp_file)

PromptTemplate(input_variables=['prompt_text'], output_parser=None, partial_variables={}, template='#命令書:\nあなたは、［プロの翻訳家］です。\n以下の条件に沿って与えられた入力文章をもとに、最高の［翻訳文］を出力してください。\nあなたが不要だと思う文章は、出力しなくても構いません。\n\n# 処理手順:\n・［入力文に対して、翻訳の初稿を作成する］\n・［翻訳文を読み返し、制約条件に合っているか確認する］\n・［翻訳文を修正し、出力する］\n\n#制約条件:\n・［日本語に翻訳する］\n・［入力文章をそのまま翻訳する］\n・［箇条書きの文章は箇条書きで出力する］\n\n#入力文：{prompt_text}\n\n#出力文：', template_format='f-string', validate_template=True)

In [32]:
translated_text = summarizer.translate(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  1243.26 ms
llama_print_timings:      sample time =    51.68 ms /    65 runs   (    0.80 ms per token,  1257.81 tokens per second)
llama_print_timings: prompt eval time =  2177.70 ms /    87 tokens (   25.03 ms per token,    39.95 tokens per second)
llama_print_timings:        eval time =  2867.72 ms /    64 runs   (   44.81 ms per token,    22.32 tokens per second)
llama_print_timings:       total time =  5446.96 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  1243.26 ms
Llama.generate: prefix-match hit
llama_print_timings:      sample time =    58.59 ms /    78 runs   (    0.75 ms per token,  1331.33 tokens per second)
llama_print_timings: prompt eval time =  1587.94 ms /    70 tokens (   22.68 ms per token,    44.08 tokens per second)
llama_print_timings:        eval time =  3491.66 ms /    77 runs   (   45.35 ms per token,    22.05 tokens per second)
llama_print_timings:       to

In [1]:
print(translated_text)

NameError: name 'translated_text' is not defined